# Building an MTCollection

The first step to analyzing a set of transfer functions is to load them into an `MTCollection` which is just a wrapper around an `MTH5` file.  The advantage to doing this is that you only have to do this once from the various transfer functions that you have, which might include various flavors of EDI, EMTFXML, J-files, Z-files, AVG-files, etc.  The other advantage is that now you have a database to work with: a single file where data is readily accessible vs. multiple ASCII files that need to be read in each time you want to do something.  

Using the class `MT` any<a name="fn1"></a>[<sup>[1]</sup>](#fn1) transfer function can be read into a generic transfer function container `MT`.  If you want to adjust `survey.id` or other metadata attributes you can do it from the `MT` object.  

<a name="fn1"></a>1. [^](#fn1) It works 100% of the time 50% of the time.  Most transfer function files are supported but if you find one is not raise an [issue](https://github.com/MTgeophysics/mtpy/issues)

## 1. Get List of Transfer Function Files

Step one is locating all the transfer function files you want to read in.  Here we will read in a couple different folders.  